## Propuesta FastText
### David Romero Grajales



In [9]:
import fasttext
import pandas as pdf
rom google.colab import drive
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/data_toxic_cleaned2.csv'
df4 = pd.read_csv(file_path, sep=',')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
df4.head()

,message,label,message_clean,message_clean2
0,"Hola, dale actualizar a la pagina o la tecla F5",0,hola dalar actualizar pagina tecla f5,hola dalar actualizar pagina tecla f5
1,Ningún mujer te va a dejar por un error. porqu...,1,ningun mujer ir dejar error humano algun momen...,ningun mujer ir dejar error humano algun momen...
2,"Entonces, si está haciendo una correspondencia...",0,entonces si hacer correspondencia r 2 dibujari...,entonces hacer correspondencia dibujaria eleme...
3,victoria con el pelito corto me voy a matar,1,victoria pelito corto ir matar,victoria pelito corto ir matar
4,@LoloPeniche Periodista!!!!!\nEs una puta verd...,1,lolopeniche periodista puta verdulero hdsptm,lolopeniche periodista puta verdulero hdsptm


Se añade el prefijo __label__ a las etiquetas y se concatenan con los mensajes para cumplir con el formato requerido por FastText, que distingue las etiquetas del texto para entrenar modelos de clasificación. Este formato estandariza los datos, facilitando el aprendizaje automático de la asociación entre textos y sus categorías correspondientes.

In [18]:
df4['label'] = '__label__' + df4['label'].astype(str)
df4['fasttext_format'] = df4['label'] + ' ' + df4['message']

# Dividir los datos en entrenamiento y prueba
from sklearn.model_selection import train_test_split

train, test = train_test_split(df4['fasttext_format'], test_size=0.2, random_state=42)

# Guardar los datos en archivos de texto
train.to_csv('train3.txt', index=False, header=False, sep='\n')
test.to_csv('test3.txt', index=False, header=False, sep='\n')

In [19]:
# Entrenar el modelo
model = fasttext.train_supervised(input='train3.txt')

# Guardar el modelo
model.save_model("modelo_fasttext3.bin")

In [21]:
model = fasttext.load_model("modelo_fasttext3.bin")

# Evaluar el modelo
result = model.test('test3.txt')
print(f"Precision: {result[1]}")
print(f"Recall: {result[2]}")

Precision: 0.9681843958225037
Recall: 0.9681843958225037


El modelo FastText alcanzó una alta precisión y recall de 0.968 en poco tiempo y sin necesidad de preprocesar los datos, demostrando su eficacia y rapidez para la clasificación de texto.

In [26]:
# Pruebas

def classify_text(text):
    labels, probabilities = model.predict([text], k=1)

    label = labels[0][0]
    probability = probabilities[0][0]

    label = label.replace("__label__", "")

    return label, probability


In [32]:
# Comentarios buenos
text = " que buena empresa, hace un gran trabajo"
label, probability = classify_text(text)
probability_rounded = round(float(probability), 2)
print(f"Etiqueta: {label}, Probabilidad: {probability_rounded}")


Etiqueta: 0, Probabilidad: 0.89


In [33]:
# Comentarios toxicos
text = "que producto de mierda"
label, probability = classify_text(text)
probability_rounded = round(float(probability), 2)
print(f"Etiqueta: {label}, Probabilidad: {probability_rounded}")


Etiqueta: 1, Probabilidad: 0.86
